# Phase 2: Feature Engineering for Commodity Price Predictor

## Objective
Engineer 100-150 features from price data and weather data for ML model training.

**Key Focus**: Temperature lag features (correlation +0.81 from Phase 1)

## Expected Output
- Engineered dataset with 100-150 features
- 13,900 rows × 100-150 columns
- ML-ready normalized data
- Feature documentation

## Timeline
Estimated: 7-12 hours for all 10 steps

## STEP 1: Environment Setup & Data Loading

In [1]:
import pandas as pd
import numpy as np
import os
import mysql.connector
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"   Pandas version: {pd.__version__}")
print(f"   NumPy version: {np.__version__}")

✅ Libraries imported successfully
   Pandas version: 2.2.3
   NumPy version: 2.3.4


### Load Price Data from Phase 1

In [2]:
# Load processed price data
processed_data_dir = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Processed_Data"
price_file = os.path.join(processed_data_dir, 'Price_Data_Processed.csv')

df_prices = pd.read_csv(price_file)
df_prices['Arrival_Date'] = pd.to_datetime(df_prices['Arrival_Date'])

print(f"✅ Price data loaded")
print(f"   Records: {len(df_prices):,}")
print(f"   Columns: {len(df_prices.columns)}")
print(f"   Date Range: {df_prices['Arrival_Date'].min().date()} to {df_prices['Arrival_Date'].max().date()}")
print(f"   Missing Values: {df_prices.isnull().sum().sum()}")
print(f"\n   Columns: {list(df_prices.columns)}")

✅ Price data loaded
   Records: 14,965
   Columns: 18
   Date Range: 2024-01-01 to 2025-01-01
   Missing Values: 0

   Columns: ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Arrival_Date', 'Min_Price', 'Max_Price', 'Modal_Price', 'Commodity_Code', 'Year', 'Month', 'Day', 'DayName', 'Quarter', 'Price_Range', 'Price_Volatility_%']


### Load Weather Data

In [3]:
# Connect to weather database
DB_HOST = os.environ.get('DB_HOST', 'localhost')
DB_USER = os.environ.get('DB_USER', 'root')
DB_PASS = os.environ.get('DB_PASS', 'root')
DB_NAME = os.environ.get('DB_NAME', 'weather_history')

try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME
    )
    df_weather = pd.read_sql("SELECT * FROM cleaned_weather_data;", conn)
    conn.close()
    
    df_weather.columns = [c.strip() for c in df_weather.columns]
    if 'time' in df_weather.columns:
        df_weather['date'] = pd.to_datetime(df_weather['time'])
    
    print(f"✅ Weather data loaded")
    print(f"   Records: {len(df_weather):,}")
    print(f"   Date Range: {df_weather['date'].min().date()} to {df_weather['date'].max().date()}")
    print(f"\n   Weather Columns: {list(df_weather.columns[:10])}")
except Exception as e:
    print(f"⚠️  Could not load weather from DB: {e}")
    df_weather = pd.DataFrame()

✅ Weather data loaded
   Records: 7,707
   Date Range: 2024-01-01 to 2025-01-01

   Weather Columns: ['market_name', 'latitude', 'longitude', 'time', 'temperature_max', 'temperature_min', 'temperature_mean', 'precipitation_sum', 'rain_sum', 'wind_speed_max']


## STEP 2: Data Merge & Aggregation

In [4]:
# Prepare price data for merging
df_prices['date'] = df_prices['Arrival_Date'].dt.date

# Create commodity-market groups
df_prices['Commodity_Market'] = df_prices['Commodity'] + '_' + df_prices['Market']

print(f"✅ Data prepared for merging")
print(f"   Unique Commodity-Market pairs: {df_prices['Commodity_Market'].nunique()}")

# Merge with weather data
if not df_weather.empty:
    df_weather['date'] = pd.to_datetime(df_weather['date']).dt.date
    
    # Create market mapping if available
    # For now, aggregate weather by date only
    df_weather_daily = df_weather.groupby('date').agg({
        'temperature_max': 'mean',
        'temperature_min': 'mean',
        'temperature_mean': 'mean',
        'precipitation_sum': 'mean',
        'rain_sum': 'mean',
        'wind_speed_max': 'mean',
        'wind_gusts_max': 'mean',
        'cloud_cover': 'mean'
    }).reset_index()
    
    # Merge price and weather
    df_combined = df_prices.merge(
        df_weather_daily,
        on='date',
        how='left'
    )
    
    print(f"✅ Data merged successfully")
    print(f"   Merged records: {len(df_combined):,}")
    print(f"   Merge success rate: {len(df_combined[df_combined['temperature_min'].notna()]) / len(df_combined) * 100:.1f}%")
else:
    df_combined = df_prices.copy()
    print(f"⚠️  Weather data unavailable, proceeding with price data only")

✅ Data prepared for merging
   Unique Commodity-Market pairs: 129
✅ Data merged successfully
   Merged records: 14,965
   Merge success rate: 100.0%


## STEP 3: Lagged Price Features

In [5]:
# Sort by commodity-market and date
df_combined = df_combined.sort_values(['Commodity_Market', 'Arrival_Date']).reset_index(drop=True)

# Create lagged price features
for lag in [1, 7, 14, 30]:
    df_combined[f'Price_Lag_{lag}'] = df_combined.groupby('Commodity_Market')['Modal_Price'].shift(lag)
    df_combined[f'Min_Price_Lag_{lag}'] = df_combined.groupby('Commodity_Market')['Min_Price'].shift(lag)
    df_combined[f'Max_Price_Lag_{lag}'] = df_combined.groupby('Commodity_Market')['Max_Price'].shift(lag)

print(f"✅ Lagged price features created")
lag_features = [col for col in df_combined.columns if 'Lag' in col]
print(f"   Features added: {len(lag_features)}")
print(f"   Example features: {lag_features[:3]}")
print(f"   NaN values in lags: {df_combined[lag_features].isnull().sum().sum():,} (first 30 rows expected)")

✅ Lagged price features created
   Features added: 12
   Example features: ['Price_Lag_1', 'Min_Price_Lag_1', 'Max_Price_Lag_1']
   NaN values in lags: 15,972 (first 30 rows expected)


## STEP 4: Rolling Statistics

In [6]:
# Rolling averages and statistics
for window in [7, 14, 30]:
    df_combined[f'Price_MA_{window}'] = df_combined.groupby('Commodity_Market')['Modal_Price'].transform(
        lambda x: x.rolling(window, min_periods=1).mean()
    )
    df_combined[f'Price_Std_{window}'] = df_combined.groupby('Commodity_Market')['Modal_Price'].transform(
        lambda x: x.rolling(window, min_periods=1).std()
    )
    df_combined[f'Price_Max_{window}'] = df_combined.groupby('Commodity_Market')['Modal_Price'].transform(
        lambda x: x.rolling(window, min_periods=1).max()
    )
    df_combined[f'Price_Min_{window}'] = df_combined.groupby('Commodity_Market')['Modal_Price'].transform(
        lambda x: x.rolling(window, min_periods=1).min()
    )

print(f"✅ Rolling statistics created")
rolling_features = [col for col in df_combined.columns if 'MA_' in col or 'Std_' in col or ('Max_' in col and 'Price' in col) or ('Min_' in col and 'Price' in col)]
print(f"   Features added: {len(rolling_features)}")
print(f"   Example features: {rolling_features[:3]}")

✅ Rolling statistics created
   Features added: 22
   Example features: ['Min_Price', 'Max_Price', 'Min_Price_Lag_1']


## STEP 5: Momentum & Trend Features

In [7]:
# Percentage changes
df_combined['Price_Change_1d_%'] = df_combined.groupby('Commodity_Market')['Modal_Price'].pct_change() * 100
df_combined['Price_Change_7d_%'] = df_combined.groupby('Commodity_Market')['Modal_Price'].pct_change(7) * 100
df_combined['Price_Change_30d_%'] = df_combined.groupby('Commodity_Market')['Modal_Price'].pct_change(30) * 100

# Price range (already exists but create normalized version)
df_combined['Price_Range_Normalized'] = (
    (df_combined['Max_Price'] - df_combined['Min_Price']) / df_combined['Min_Price'].replace(0, 1) * 100
)

print(f"✅ Momentum features created")
momentum_features = [col for col in df_combined.columns if 'Change' in col or 'Range_Normalized' in col]
print(f"   Features added: {len(momentum_features)}")
print(f"   Example: {momentum_features[:2]}")
print(f"   NaN count: {df_combined[momentum_features].isnull().sum().sum():,}")

✅ Momentum features created
   Features added: 4
   Example: ['Price_Change_1d_%', 'Price_Change_7d_%']
   NaN count: 3,821


## STEP 6: Calendar & Seasonal Features

In [8]:
# Extract temporal features (already present, but add more)
df_combined['Day_of_Week'] = df_combined['Arrival_Date'].dt.dayofweek
df_combined['Day_of_Year'] = df_combined['Arrival_Date'].dt.dayofyear
df_combined['Week_of_Year'] = df_combined['Arrival_Date'].dt.isocalendar().week
df_combined['Is_Weekend'] = (df_combined['Day_of_Week'] >= 5).astype(int)

# Seasonal indicators (India agricultural seasons)
def get_season(month):
    if month in [12, 1, 2]:
        return 1  # Winter
    elif month in [3, 4, 5]:
        return 2  # Summer
    elif month in [6, 7, 8, 9]:
        return 3  # Monsoon
    else:
        return 4  # Post-Monsoon

df_combined['Season'] = df_combined['Month'].apply(get_season)

# Cyclical encoding (sine/cosine for circular features)
df_combined['Month_Sin'] = np.sin(2 * np.pi * df_combined['Month'] / 12)
df_combined['Month_Cos'] = np.cos(2 * np.pi * df_combined['Month'] / 12)
df_combined['DayOfWeek_Sin'] = np.sin(2 * np.pi * df_combined['Day_of_Week'] / 7)
df_combined['DayOfWeek_Cos'] = np.cos(2 * np.pi * df_combined['Day_of_Week'] / 7)

print(f"✅ Calendar & seasonal features created")
seasonal_features = [col for col in df_combined.columns if 'Day_of' in col or 'Week' in col or 'Weekend' in col or 'Season' in col or 'Sin' in col or 'Cos' in col]
print(f"   Features added: {len(seasonal_features)}")
print(f"   Example features: {seasonal_features[:5]}")

✅ Calendar & seasonal features created
   Features added: 9
   Example features: ['Day_of_Week', 'Day_of_Year', 'Week_of_Year', 'Is_Weekend', 'Season']


## STEP 7: Weather Features & Lags (CRITICAL)

In [9]:
if not df_weather.empty and 'temperature_min' in df_combined.columns:
    print(f"✅ Creating weather features (Temperature Correlation: +0.81)")
    
    # Temperature features with lags (CRITICAL - correlation +0.81)
    for lag in [0, 1, 3, 7]:
        if lag == 0:
            df_combined['Temp_Min'] = df_combined['temperature_min']
            df_combined['Temp_Max'] = df_combined['temperature_max']
            df_combined['Temp_Mean'] = df_combined['temperature_mean']
        else:
            df_combined[f'Temp_Min_Lag_{lag}'] = df_combined['temperature_min'].shift(lag)
            df_combined[f'Temp_Max_Lag_{lag}'] = df_combined['temperature_max'].shift(lag)
            df_combined[f'Temp_Mean_Lag_{lag}'] = df_combined['temperature_mean'].shift(lag)
    
    # Temperature rolling statistics
    for window in [7, 14, 30]:
        df_combined[f'Temp_Min_MA_{window}'] = df_combined['temperature_min'].rolling(window, min_periods=1).mean()
        df_combined[f'Temp_Max_MA_{window}'] = df_combined['temperature_max'].rolling(window, min_periods=1).mean()
        df_combined[f'Temp_Range_{window}'] = (
            df_combined['temperature_max'].rolling(window, min_periods=1).max() -
            df_combined['temperature_min'].rolling(window, min_periods=1).min()
        )
    
    # Precipitation features
    if 'precipitation_sum' in df_combined.columns:
        df_combined['Rainfall'] = df_combined['precipitation_sum']
        df_combined['Is_Rainy'] = (df_combined['precipitation_sum'] > 1).astype(int)
        df_combined['Rainfall_Lag_1'] = df_combined['precipitation_sum'].shift(1)
        df_combined['Rainfall_Lag_7'] = df_combined['precipitation_sum'].shift(7)
        
        for window in [7, 30]:
            df_combined[f'Rainfall_Sum_{window}'] = df_combined['precipitation_sum'].rolling(window, min_periods=1).sum()
    
    # Wind features
    if 'wind_speed_max' in df_combined.columns:
        df_combined['Wind_Speed'] = df_combined['wind_speed_max']
        df_combined['Wind_Speed_Lag_1'] = df_combined['wind_speed_max'].shift(1)
        df_combined['Wind_Speed_MA_7'] = df_combined['wind_speed_max'].rolling(7, min_periods=1).mean()
    
    # Cloud cover
    if 'cloud_cover' in df_combined.columns:
        df_combined['Cloud_Cover'] = df_combined['cloud_cover']
        df_combined['Cloud_Cover_Lag_1'] = df_combined['cloud_cover'].shift(1)
    
    weather_features = [col for col in df_combined.columns if 'Temp' in col or 'Wind' in col or 'Cloud' in col or 'Rainfall' in col]
    print(f"   Weather features added: {len(weather_features)}")
    print(f"   Example features: {weather_features[:5]}")
else:
    print(f"⚠️  Weather data unavailable for feature engineering")
    weather_features = []

✅ Creating weather features (Temperature Correlation: +0.81)
   Weather features added: 31
   Example features: ['Temp_Min', 'Temp_Max', 'Temp_Mean', 'Temp_Min_Lag_1', 'Temp_Max_Lag_1']


## STEP 8: Commodity-Market Business Features

In [10]:
# Historical commodity statistics
commodity_stats = df_combined.groupby('Commodity').agg({
    'Modal_Price': ['mean', 'std', 'min', 'max'],
    'Price_Volatility_%': 'mean'
}).reset_index()

commodity_stats.columns = ['Commodity', 'Commodity_Avg_Price', 'Commodity_Std_Price',
                           'Commodity_Min_Price', 'Commodity_Max_Price', 'Commodity_Volatility']

df_combined = df_combined.merge(commodity_stats, on='Commodity', how='left')

# Historical market statistics
market_stats = df_combined.groupby('Market').agg({
    'Modal_Price': ['mean', 'std'],
    'Commodity': 'nunique'
}).reset_index()

market_stats.columns = ['Market', 'Market_Avg_Price', 'Market_Std_Price', 'Market_Commodity_Count']
df_combined = df_combined.merge(market_stats, on='Market', how='left')

# Commodity-Market interaction
comm_market_stats = df_combined.groupby(['Commodity', 'Market']).agg({
    'Modal_Price': ['mean', 'std', 'count']
}).reset_index()

comm_market_stats.columns = ['Commodity', 'Market', 'Comm_Market_Avg_Price',
                             'Comm_Market_Std_Price', 'Comm_Market_Records']

df_combined = df_combined.merge(comm_market_stats, on=['Commodity', 'Market'], how='left')

# Seasonal commodity premium
seasonal_premium = df_combined.groupby(['Month', 'Commodity'])['Modal_Price'].mean().reset_index()
seasonal_premium.columns = ['Month', 'Commodity', 'Month_Commodity_Avg_Price']
df_combined = df_combined.merge(seasonal_premium, on=['Month', 'Commodity'], how='left')

print(f"✅ Business features created")
business_features = [col for col in df_combined.columns if any(x in col for x in ['Commodity_Avg', 'Market_Avg', 'Comm_Market', 'Month_Commodity'])]
print(f"   Features added: {len(business_features)}")
print(f"   Example features: {business_features[:5]}")

✅ Business features created
   Features added: 6
   Example features: ['Commodity_Avg_Price', 'Market_Avg_Price', 'Comm_Market_Avg_Price', 'Comm_Market_Std_Price', 'Comm_Market_Records']


## STEP 9: Categorical Encoding

In [11]:
# One-hot encoding for Grade
df_combined = pd.get_dummies(df_combined, columns=['Grade'], drop_first=True, prefix='Grade')

# One-hot encoding for DayName
if 'DayName' in df_combined.columns:
    df_combined = pd.get_dummies(df_combined, columns=['DayName'], drop_first=True, prefix='Day')

# Target encoding for Commodity
commodity_encoding = df_combined.groupby('Commodity')['Modal_Price'].mean()
df_combined['Commodity_Encoded'] = df_combined['Commodity'].map(commodity_encoding)

# Target encoding for Market
market_encoding = df_combined.groupby('Market')['Modal_Price'].mean()
df_combined['Market_Encoded'] = df_combined['Market'].map(market_encoding)

print(f"✅ Categorical encoding completed")
encoding_features = [col for col in df_combined.columns if 'Grade_' in col or 'Day_' in col or 'Encoded' in col]
print(f"   Encoding features added: {len(encoding_features)}")
print(f"   Example features: {encoding_features[:5]}")

✅ Categorical encoding completed
   Encoding features added: 12
   Example features: ['Day_of_Week', 'Day_of_Year', 'Grade_Medium', 'Grade_Small', 'Day_Monday']


## STEP 10: Data Validation & Normalization

In [13]:
# Drop first 30 rows (lag values not available)
print(f"Dataset before dropping NaN rows: {len(df_combined):,}")
df_combined = df_combined.iloc[30:].reset_index(drop=True)
print(f"Dataset after dropping first 30 rows: {len(df_combined):,}")

# Handle remaining NaNs
nan_counts = df_combined.isnull().sum()
nan_cols = nan_counts[nan_counts > 0]

if len(nan_cols) > 0:
    print(f"\nColumns with NaN values:")
    for col, count in nan_cols.items():
        print(f"  {col}: {count:,} ({count/len(df_combined)*100:.1f}%)")
    
    # Forward fill then backward fill
    df_combined = df_combined.fillna(method='ffill').fillna(method='bfill')
else:
    print(f"\n✅ No remaining NaN values")

# Replace infinite values
df_combined = df_combined.replace([np.inf, -np.inf], np.nan)

# Fill remaining NaNs with mean of numeric columns only
numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_combined[col].isnull().any():
        df_combined[col].fillna(df_combined[col].mean(), inplace=True)

print(f"\n✅ Data cleaned")
print(f"   Final shape: {df_combined.shape}")
print(f"   Remaining NaN: {df_combined.isnull().sum().sum()}")

Dataset before dropping NaN rows: 14,935
Dataset after dropping first 30 rows: 14,905

✅ No remaining NaN values

✅ Data cleaned
   Final shape: (14905, 117)
   Remaining NaN: 0


### Feature Normalization

In [14]:
# Separate features and target
drop_cols = ['Commodity', 'Market', 'Variety', 'Arrival_Date', 'date', 'Commodity_Market',
             'Min_Price', 'Max_Price', 'Arrival']

feature_cols = [col for col in df_combined.columns if col not in drop_cols + ['Modal_Price']]
X = df_combined[feature_cols].copy()
y = df_combined['Modal_Price'].copy()

print(f"✅ Features separated from target")
print(f"   Feature columns: {len(feature_cols)}")
print(f"   Target variable: Modal_Price")
print(f"   Target shape: {y.shape}")

# Normalize numeric features
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

print(f"\n✅ Features normalized (StandardScaler)")
print(f"   Numeric features normalized: {len(numeric_features)}")
print(f"   Sample statistics after scaling:")
print(f"      Mean: {X[numeric_features].mean().mean():.6f}")
print(f"      Std: {X[numeric_features].std().mean():.6f}")

# Save scaler
scaler_file = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\scaler.pkl"
with open(scaler_file, 'wb') as f:
    pickle.dump(scaler, f)
print(f"\n✅ Scaler saved to: {scaler_file}")

✅ Features separated from target
   Feature columns: 108
   Target variable: Modal_Price
   Target shape: (14905,)

✅ Features normalized (StandardScaler)
   Numeric features normalized: 98
   Sample statistics after scaling:
      Mean: 0.000000
      Std: 1.000034

✅ Scaler saved to: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\scaler.pkl


## STEP 11: Save Engineered Dataset

In [15]:
# Create final dataset
df_final = pd.concat([X, y.reset_index(drop=True)], axis=1)
df_final['Arrival_Date'] = df_combined['Arrival_Date'].values
df_final['Commodity'] = df_combined['Commodity'].values
df_final['Market'] = df_combined['Market'].values

# Save engineered dataset
output_file = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Processed_Data\Features_Engineered.csv"
df_final.to_csv(output_file, index=False)

print(f"✅ Engineered dataset saved")
print(f"   File: {output_file}")
print(f"   Shape: {df_final.shape}")
print(f"   Records: {len(df_final):,}")
print(f"   Features: {len(feature_cols)}")

✅ Engineered dataset saved
   File: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Processed_Data\Features_Engineered.csv
   Shape: (14905, 112)
   Records: 14,905
   Features: 108


## STEP 12: Feature Documentation & Summary

In [16]:
# Create feature documentation
feature_types = {
    'Lagged Price Features': [col for col in feature_cols if 'Lag' in col and 'Price' in col],
    'Rolling Statistics': [col for col in feature_cols if 'MA_' in col or 'Std_' in col],
    'Momentum Features': [col for col in feature_cols if 'Change' in col],
    'Seasonal Features': [col for col in feature_cols if any(x in col for x in ['Season', 'Sin', 'Cos', 'Week'])],
    'Weather Features': [col for col in feature_cols if any(x in col for x in ['Temp', 'Wind', 'Cloud', 'Rainfall'])],
    'Business Features': [col for col in feature_cols if any(x in col for x in ['Commodity_Avg', 'Market_Avg', 'Comm_Market'])],
    'Categorical Features': [col for col in feature_cols if 'Grade_' in col or 'Day_' in col or 'Encoded' in col]
}

print(f"\n" + "="*80)
print(f"FEATURE ENGINEERING SUMMARY")
print(f"="*80)
print(f"\nTotal Features: {len(feature_cols)}\n")

for category, cols in feature_types.items():
    print(f"{category}: {len(cols)} features")
    if len(cols) > 0:
        print(f"  Examples: {cols[:3]}")
print(f"\n" + "="*80)
print(f"Dataset Statistics:")
print(f"  Records: {len(df_final):,}")
print(f"  Features: {len(feature_cols)}")
print(f"  Missing Values: {df_final.isnull().sum().sum()}")
print(f"  Target Variable: Modal_Price (Rs/Quintal)")
print(f"  Ready for ML: YES ✓")
print(f"="*80)


FEATURE ENGINEERING SUMMARY

Total Features: 108

Lagged Price Features: 12 features
  Examples: ['Price_Lag_1', 'Min_Price_Lag_1', 'Max_Price_Lag_1']
Rolling Statistics: 16 features
  Examples: ['Price_MA_7', 'Price_Std_7', 'Price_MA_14']
Momentum Features: 3 features
  Examples: ['Price_Change_1d_%', 'Price_Change_7d_%', 'Price_Change_30d_%']
Seasonal Features: 8 features
  Examples: ['Day_of_Week', 'Week_of_Year', 'Is_Weekend']
Weather Features: 31 features
  Examples: ['Temp_Min', 'Temp_Max', 'Temp_Mean']
Business Features: 6 features
  Examples: ['Commodity_Avg_Price', 'Market_Avg_Price', 'Comm_Market_Avg_Price']
Categorical Features: 12 features
  Examples: ['Day_of_Week', 'Day_of_Year', 'Grade_Medium']

Dataset Statistics:
  Records: 14,905
  Features: 108
  Missing Values: 0
  Target Variable: Modal_Price (Rs/Quintal)
  Ready for ML: YES ✓


## STEP 13: Feature Validation & Correlation Analysis

In [18]:
# Compute correlations with target (numeric features only)
X_numeric = X.select_dtypes(include=[np.number])
correlations = X_numeric.corrwith(y).sort_values(ascending=False)

print(f"\nTop 15 Features by Correlation with Modal_Price:")
print(correlations.head(15))

print(f"\n\nBottom 15 Features by Correlation with Modal_Price:")
print(correlations.tail(15))

# Save correlations
corr_file = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Processed_Data\Feature_Correlations.csv"
correlations.to_csv(corr_file, header=False)
print(f"\n✅ Feature correlations saved to: {corr_file}")


Top 15 Features by Correlation with Modal_Price:
Price_MA_7                   0.972150
Price_Lag_1                  0.962959
Price_Max_7                  0.958074
Month_Commodity_Avg_Price    0.955239
Price_MA_14                  0.951207
Price_Min_7                  0.948516
Price_Max_14                 0.930255
Max_Price_Lag_1              0.930237
Price_MA_30                  0.923496
Price_Min_14                 0.910769
Min_Price_Lag_1              0.890732
Price_Max_30                 0.889585
Price_Lag_7                  0.874406
Price_Min_30                 0.862125
Comm_Market_Avg_Price        0.861518
dtype: float64


Bottom 15 Features by Correlation with Modal_Price:
Is_Weekend                0.008910
DayOfWeek_Cos             0.002837
Year                     -0.004766
DayOfWeek_Sin            -0.007478
Day                      -0.011166
Market_Commodity_Count   -0.027595
Temp_Range_30            -0.044004
Month_Sin                -0.060101
Temp_Range_14            -0.066

## FINAL STATUS

In [19]:
print(f"""
╔════════════════════════════════════════════════════════════════╗
║           PHASE 2: FEATURE ENGINEERING COMPLETE              ║
╚════════════════════════════════════════════════════════════════╝

✅ DELIVERABLES:
   • Features_Engineered.csv - {len(df_final):,} records × {len(feature_cols)} features
   • Feature_Correlations.csv - Correlation analysis
   • scaler.pkl - StandardScaler for inference

📊 FEATURE BREAKDOWN:
   • Lagged Features: {len(feature_types['Lagged Price Features'])}
   • Rolling Statistics: {len(feature_types['Rolling Statistics'])}
   • Momentum Features: {len(feature_types['Momentum Features'])}
   • Seasonal Features: {len(feature_types['Seasonal Features'])}
   • Weather Features: {len(feature_types['Weather Features'])}
   • Business Features: {len(feature_types['Business Features'])}
   • Categorical Features: {len(feature_types['Categorical Features'])}
   ─────────────────────────────────────────
   • TOTAL: {len(feature_cols)} features

🎯 DATA QUALITY:
   • Records: {len(df_final):,}
   • Missing Values: {df_final.isnull().sum().sum()}
   • Normalized: YES ✓
   • Ready for ML: YES ✓

⏭️  NEXT: Phase 3 - ML Model Development
   Start with: PHASE_3_ML_Models.ipynb

╔════════════════════════════════════════════════════════════════╗
║    🚀 PHASE 2 COMPLETE - READY FOR ML MODEL TRAINING 🚀      ║
╚════════════════════════════════════════════════════════════════╝
""")


╔════════════════════════════════════════════════════════════════╗
║           PHASE 2: FEATURE ENGINEERING COMPLETE              ║
╚════════════════════════════════════════════════════════════════╝

✅ DELIVERABLES:
   • Features_Engineered.csv - 14,905 records × 108 features
   • Feature_Correlations.csv - Correlation analysis
   • scaler.pkl - StandardScaler for inference

📊 FEATURE BREAKDOWN:
   • Lagged Features: 12
   • Rolling Statistics: 16
   • Momentum Features: 3
   • Seasonal Features: 8
   • Weather Features: 31
   • Business Features: 6
   • Categorical Features: 12
   ─────────────────────────────────────────
   • TOTAL: 108 features

🎯 DATA QUALITY:
   • Records: 14,905
   • Missing Values: 0
   • Normalized: YES ✓
   • Ready for ML: YES ✓

⏭️  NEXT: Phase 3 - ML Model Development
   Start with: PHASE_3_ML_Models.ipynb

╔════════════════════════════════════════════════════════════════╗
║    🚀 PHASE 2 COMPLETE - READY FOR ML MODEL TRAINING 🚀      ║
╚═════════════════════